In [1]:
# Gerekli Kütüphanelerin Yüklenmesi
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, udf, explode, collect_list
from pyspark.sql.types import IntegerType, FloatType, StringType, StructType, StructField, ArrayType

# Makine Öğrenmesi - İşbirlikçi Filtreleme (ALS)
# sana benzeyen kullanıcılar neleri sevdi
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

# Makine Öğrenmesi - İçerik Tabanlı Filtreleme (Cosine Similarity)
# Filmlerin türlerini (genres) kullanarak bir filmi seven kullanıcıya, o filme içerik olarak en çok benzeyen diğer filmleri 
# Kosinüs Benzerliği (Cosine Similarity) ile önereceğiz.
from pyspark.ml.feature import CountVectorizer, IDF, Normalizer
from pyspark.ml.linalg import SparseVector
from pyspark.ml import Pipeline

import findspark
import pandas as pd
import numpy as np
import time

# findspark.init() # Eğer Spark'ı otomatik bulamazsa bu satırı aktifleştir

spark = (SparkSession.builder
    .appName("KapsamliFilmTavsiyeSistemi")
    # Büyük veri seti için bellek ayarlarını artırıyoruz
    .config("spark.driver.memory", "8g")
    .config("spark.executor.memory", "8g")
    # Verimli shuffle işlemleri için
    .config("spark.sql.shuffle.partitions", "200")
    # Yerel makinenin tüm çekirdeklerini kullanmamızı sağlar
    .master("local[*]")
    .getOrCreate()
)

sc = spark.sparkContext
sc.setLogLevel("WARN")

print(f"SparkSession başlatıldı. Spark Sürümü: {spark.version}")
print(f"SparkContext mevcut: {sc.appName}")

25/07/01 11:33:38 WARN Utils: Your hostname, turan-Dell-G15-5511 resolves to a loopback address: 127.0.1.1; using 172.17.30.206 instead (on interface wlp0s20f3)
25/07/01 11:33:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/01 11:33:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


SparkSession başlatıldı. Spark Sürümü: 3.5.1
SparkContext mevcut: KapsamliFilmTavsiyeSistemi


In [2]:
# Dosya Yolları ve Şemaların Tanımlanması

BASE_PATH = "./"
MOVIES_PATH = f"{BASE_PATH}movies.csv"
RATINGS_PATH = f"{BASE_PATH}ratings.csv"
GENOME_SCORES_PATH = f"{BASE_PATH}genome-scores.csv"
GENOME_TAGS_PATH = f"{BASE_PATH}genome-tags.csv"

movie_schema = StructType([
    StructField("movieId", IntegerType(), True),
    StructField("title", StringType(), True),
    StructField("genres", StringType(), True)
])

rating_schema = StructType([
    StructField("userId", IntegerType(), True),
    StructField("movieId", IntegerType(), True),
    StructField("rating", FloatType(), True),
    StructField("timestamp", IntegerType(), True)
])

genome_scores_schema = StructType([
    StructField("movieId", IntegerType(), True),
    StructField("tagId", IntegerType(), True),
    StructField("relevance", FloatType(), True)
])

genome_tags_schema = StructType([
    StructField("tagId", IntegerType(), True),
    StructField("tag", StringType(), True)
])

print("CSV dosyaları yükleniyor...")
movies_df = spark.read.csv(MOVIES_PATH, header=True, schema=movie_schema, escape='"')
ratings_df_raw = spark.read.csv(RATINGS_PATH, header=True, schema=rating_schema)
genome_scores_df = spark.read.csv(GENOME_SCORES_PATH, header=True, schema=genome_scores_schema)
genome_tags_df = spark.read.csv(GENOME_TAGS_PATH, header=True, schema=genome_tags_schema)

movies_df.cache()
ratings_df_raw.cache()
genome_scores_df.cache()
genome_tags_df.cache()

print("\nTüm veri setleri başarıyla yüklendi.")

CSV dosyaları yükleniyor...

Tüm veri setleri başarıyla yüklendi.


In [3]:
# Veri Ön İşleme (ALS ve Kosinüs Benzerliği için Hazırlık)

# --- 5. ALS için Veri Hazırlığı ---
als_ratings_df = ratings_df_raw.select(
    col("userId").alias("user"),
    col("movieId").alias("item"),
    col("rating")
).cache()
print("\nALS modeli için hazırlanan DataFrame hazır.")

# --- 6. ZENGİNLEŞTİRİLMİŞ Kosinüs Benzerliği için Veri Hazırlığı (TAG GENOME KULLANARAK) ---
# genome-scores'u her film için bir vektöre dönüştüreceğiz.
# (movieId, [relevance1, relevance2, ...]) formatına getiriyoruz.
# Önemli: tagId'lerin sıralı olduğunu varsayıyoruz (1'den n'e kadar).

from pyspark.ml.linalg import Vectors, VectorUDT

# Pivoting işlemi, her movieId için tüm tag relevance değerlerini bir satırda toplar.
# Bu işlem büyük veri setlerinde yavaş olabilir, ama genome için yönetilebilir.
movies_tag_genome_df = (genome_scores_df
                        .groupBy("movieId")
                        .agg(collect_list("relevance").alias("relevance_list"))
                        .orderBy("movieId"))

# Listeyi bir Spark Vektörüne çeviren UDF
list_to_vector_udf = udf(lambda l: Vectors.dense(sorted(l)), VectorUDT())

# Vektörleri oluştur
movies_featured_df = movies_tag_genome_df.withColumn(
    "features",
    list_to_vector_udf(col("relevance_list"))
).select("movieId", "features").cache()

# Normalizer (isteğe bağlı ama önerilir)
normalizer = Normalizer(inputCol="features", outputCol="normFeatures")
movies_featured_df = normalizer.transform(movies_featured_df).selectExpr("movieId", "normFeatures as features")


print("\nİçerik-tabanlı filtreleme için TAG GENOME tabanlı özellik vektörleri oluşturuldu.")
movies_featured_df.show(5, truncate=False)


ALS modeli için hazırlanan DataFrame hazır.

İçerik-tabanlı filtreleme için TAG GENOME tabanlı özellik vektörleri oluşturuldu.


[Stage 8:====================================================>  (192 + 8) / 200]

+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [14]:
# Yöntem 1 - ALS Modelini Eğitme ve Değerlendirme (Eğitim Seti RMSE'si ile Birlikte)

# --- 7. Veri Setini Ayırma (Eğitim ve Test) ---
(training_df, test_df) = als_ratings_df.randomSplit([0.8, 0.2], seed=42)
training_df.cache()
test_df.cache()
print(f"Eğitim seti boyutu: {training_df.count()}")
print(f"Test seti boyutu: {test_df.count()}")


# --- 8. ALS Modelini Eğitme ---
# Parametreler (Sizin seçtiğiniz yüksek değerler)
rank_param = 50
max_iter_param = 15
reg_param = 0.1

als = ALS(
    userCol="user",
    itemCol="item",
    ratingCol="rating",
    rank=rank_param,
    maxIter=max_iter_param,
    regParam=reg_param,
    coldStartStrategy="drop",
    nonnegative=True
)

print("\nALS modeli eğitiliyor...")
start_time = time.time()
als_model = als.fit(training_df)
end_time = time.time()
print(f"ALS modeli {end_time - start_time:.2f} saniyede eğitildi.")


# --- 9. Modeli Değerlendirme ---
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

# 9.1. Eğitim Seti Üzerindeki Performans
print("\nModel eğitim verisi üzerinde değerlendiriliyor...")
train_predictions = als_model.transform(training_df)
train_rmse = evaluator.evaluate(train_predictions)
print(f"Eğitim Seti Üzerindeki Kök Ortalama Kare Hatası (RMSE) = {train_rmse:.4f}")

# 9.2. Test Seti Üzerindeki Performans
print("\nModel test verisi üzerinde değerlendiriliyor...")
test_predictions = als_model.transform(test_df)
test_rmse = evaluator.evaluate(test_predictions)
print(f"Test Seti Üzerindeki Kök Ortalama Kare Hatası (RMSE) = {test_rmse:.4f}")

# 9.3. Overfitting Analizi
overfitting_ratio = (test_rmse - train_rmse) / train_rmse * 100
print(f"\nTest hatası, eğitim hatasından %{overfitting_ratio:.2f} daha yüksek.")
if overfitting_ratio > 10:
    print("DİKKAT: Modelde overfitting (aşırı öğrenme) eğilimi olabilir.")

25/07/01 12:23:08 WARN CacheManager: Asked to cache already cached data.
25/07/01 12:23:08 WARN CacheManager: Asked to cache already cached data.


Eğitim seti boyutu: 27063996
Test seti boyutu: 6768166

ALS modeli eğitiliyor...


25/07/01 12:24:59 ERROR DiskBlockObjectWriter: Exception occurred while manually close the output stream to file /tmp/blockmgr-bab3a54f-9376-477c-b798-8767fa53c1a7/3d/temp_shuffle_06605942-100b-40da-8a6d-4771b4bd3a44, No space left on device
25/07/01 12:24:59 ERROR DiskBlockObjectWriter: Exception occurred while manually close the output stream to file /tmp/blockmgr-bab3a54f-9376-477c-b798-8767fa53c1a7/04/temp_shuffle_ff6e2907-890e-474c-a1b4-c09d82c4a879, No space left on device
25/07/01 12:24:59 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-bab3a54f-9376-477c-b798-8767fa53c1a7/01/temp_shuffle_9a2ad9c4-0d72-4793-87cc-70c5a452933b
25/07/01 12:24:59 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-bab3a54f-9376-477c-b798-8767fa53c1a7/0f/temp_shuffle_8348ae0b-d29d-4eab-b0ae-6a013fff90e1
25/07/01 12:24:59 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-bab3a54f-9376-477c-b798-8767fa53c1a7/1a/temp_shuffle_82fbedd6-4e33-4c44-8dcc-a8df0b475733
25/07/01 12:24:59 WAR

Py4JJavaError: An error occurred while calling o863.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 8 in stage 696.0 failed 1 times, most recent failure: Lost task 8.0 in stage 696.0 (TID 3114) (172.17.30.206 executor driver): java.io.IOException: No space left on device
	at java.base/java.io.FileOutputStream.writeBytes(Native Method)
	at java.base/java.io.FileOutputStream.write(FileOutputStream.java:349)
	at org.apache.spark.storage.TimeTrackingOutputStream.write(TimeTrackingOutputStream.java:59)
	at org.apache.spark.io.MutableCheckedOutputStream.write(MutableCheckedOutputStream.scala:43)
	at java.base/java.io.BufferedOutputStream.flushBuffer(BufferedOutputStream.java:81)
	at java.base/java.io.BufferedOutputStream.write(BufferedOutputStream.java:127)
	at net.jpountz.lz4.LZ4BlockOutputStream.flushBufferedData(LZ4BlockOutputStream.java:225)
	at net.jpountz.lz4.LZ4BlockOutputStream.write(LZ4BlockOutputStream.java:178)
	at java.base/java.io.ObjectOutputStrea

25/07/01 12:25:00 WARN TaskSetManager: Lost task 6.0 in stage 696.0 (TID 3112) (172.17.30.206 executor driver): TaskKilled (Stage cancelled: Job aborted due to stage failure: Task 8 in stage 696.0 failed 1 times, most recent failure: Lost task 8.0 in stage 696.0 (TID 3114) (172.17.30.206 executor driver): java.io.IOException: No space left on device
	at java.base/java.io.FileOutputStream.writeBytes(Native Method)
	at java.base/java.io.FileOutputStream.write(FileOutputStream.java:349)
	at org.apache.spark.storage.TimeTrackingOutputStream.write(TimeTrackingOutputStream.java:59)
	at org.apache.spark.io.MutableCheckedOutputStream.write(MutableCheckedOutputStream.scala:43)
	at java.base/java.io.BufferedOutputStream.flushBuffer(BufferedOutputStream.java:81)
	at java.base/java.io.BufferedOutputStream.write(BufferedOutputStream.java:127)
	at net.jpountz.lz4.LZ4BlockOutputStream.flushBufferedData(LZ4BlockOutputStream.java:225)
	at net.jpountz.lz4.LZ4BlockOutputStream.write(LZ4BlockOutputStream.

In [13]:
# Yöntem 1 - ALS Modelini Eğitme ve Değerlendirme

(training_df, test_df) = als_ratings_df.randomSplit([0.8, 0.2], seed=42)
training_df.cache()
test_df.cache()
print(f"Eğitim seti boyutu: {training_df.count()}")
print(f"Test seti boyutu: {test_df.count()}")

# Parametreler (Bunlar daha sonra optimize edilebilir)
rank_param = 50
max_iter_param = 15
reg_param = 0.1

als = ALS(
    userCol="user",
    itemCol="item",
    ratingCol="rating",
    rank=rank_param,
    maxIter=max_iter_param,
    regParam=reg_param,
    coldStartStrategy="drop",
    nonnegative=True
)

print("\nALS modeli eğitiliyor...")
start_time = time.time()
als_model = als.fit(training_df)
end_time = time.time()
print(f"ALS modeli {end_time - start_time:.2f} saniyede eğitildi.")

print("\nModel test verisi üzerinde değerlendiriliyor...")
predictions = als_model.transform(test_df)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(f"Test Seti Üzerindeki Kök Ortalama Kare Hatası (RMSE) = {rmse:.4f}")

25/07/01 12:16:47 WARN CacheManager: Asked to cache already cached data.
25/07/01 12:16:47 WARN CacheManager: Asked to cache already cached data.


Eğitim seti boyutu: 27063996
Test seti boyutu: 6768166

ALS modeli eğitiliyor...


ALS modeli 212.19 saniyede eğitildi.

Model test verisi üzerinde değerlendiriliyor...


[Stage 662:====================================================>  (19 + 1) / 20]

Test Seti Üzerindeki Kök Ortalama Kare Hatası (RMSE) = 0.8078


In [5]:
# Yöntem 2 - Kosinüs Benzerliği Fonksiyonu

# HÜCRE 5 - GÜNCEL HALİ

# --- 10. Kosinüs Benzerliği ile Tavsiye Fonksiyonu ---
def find_similar_movies(movie_id, top_n=10):
    """
    Verilen bir film ID'sine kosinüs benzerliğine göre en çok benzeyen `top_n` filmi döndürür.
    Bu fonksiyon artık film detaylarını da içeren bir DataFrame döndürür.
    """
    try:
        # 1. Hedef filmin özellik vektörünü bul
        # movies_featured_df'in 'features' ve 'movieId' sütunlarını içerdiğini varsayıyoruz.
        target_vector_row = movies_featured_df.filter(col("movieId") == movie_id).first()
        if not target_vector_row:
            print(f"Uyarı: {movie_id} ID'li film için özellik vektörü bulunamadı.")
            return None
        target_vector = target_vector_row.features

        # 2. Tüm filmlerle kosinüs benzerliğini hesapla (dot product)
        dot_product_udf = udf(lambda x: float(x.dot(target_vector)), FloatType())

        similarities_df = movies_featured_df.withColumn(
            "similarity",
            dot_product_udf(col("features"))
        )

        # 3. En benzer filmleri sırala, hedef filmi çıkar ve film detaylarını ekle
        top_similar_movies = (similarities_df
                              .filter(col("movieId") != movie_id)
                              .orderBy(col("similarity").desc())
                              .limit(top_n)
                              # ÖNEMLİ DÜZELTME: Film detaylarını burada ekliyoruz.
                              .join(movies_df, "movieId"))

        return top_similar_movies

    except Exception as e:
        print(f"Benzer filmler bulunurken hata: {e}")
        return None

# Fonksiyonu test edelim
print("\nKosinüs Benzerliği fonksiyonu testi: 'Toy Story (1995)' (ID: 1) için benzer filmler")
similar_to_toy_story = find_similar_movies(movie_id=1, top_n=5)
if similar_to_toy_story:
    # Fonksiyon zaten gerekli sütunları içerdiği için direkt gösterebiliriz.
    similar_to_toy_story.select("title", "genres", "similarity").show(truncate=False)


Kosinüs Benzerliği fonksiyonu testi: 'Toy Story (1995)' (ID: 1) için benzer filmler


[Stage 209:===================================================> (196 + 4) / 200]

+------------------------+--------------------------------------------------+----------+
|title                   |genres                                            |similarity|
+------------------------+--------------------------------------------------+----------+
|Fugitive, The (1993)    |Thriller                                          |0.99977887|
|Midnight Run (1988)     |Action|Comedy|Crime|Thriller                      |0.99979   |
|Gangs of New York (2002)|Crime|Drama                                       |0.999778  |
|Apocalypto (2006)       |Adventure|Drama|Thriller                          |0.9998268 |
|The Lego Movie (2014)   |Action|Adventure|Animation|Children|Comedy|Fantasy|0.9997697 |
+------------------------+--------------------------------------------------+----------+



In [11]:
# Gerekli fonksiyonları import edelim (eğer daha önce edilmediyse)
from pyspark.sql.functions import rand

# --- 11. İnteraktif Tavsiye Sistemi (Çoklu Oylama ve Çeşitlendirme) ---
print("\n" + "="*50)
print("İNTERAKTİF FİLM TAVSİYE SİSTEMİNE HOŞ GELDİNİZ")
print("="*50)

# Yeni kullanıcı için ID belirle
try:
    # Bu DataFrame'lerin var olduğunu varsayıyoruz, yoksa hata verecektir
    new_user_id = ratings_df_raw.selectExpr("max(userId)").first()[0] + 1
    print(f"Size geçici olarak '{new_user_id}' kullanıcı ID'si atanmıştır.")

    # Kullanıcıdan oylayacağı film sayısını al
    new_user_ratings = []
    num_movies_to_rate = int(input("\nKaç adet film oylamak istersiniz? (Öneri: 5-10 arası): "))
    if num_movies_to_rate <= 0:
        raise ValueError

    # Belirtilen sayıda film için oylama döngüsü
    for i in range(num_movies_to_rate):
        while True: # Kullanıcı geçerli bir film seçene kadar tekrar et
            print(f"\n--- {i+1}. Filmi Oylama ---")
            search_term = input(f"Lütfen {i+1}. filmin adını girin (örn: 'Jumanji'): ").lower()
            
            matching_movies_df = movies_df.filter(col("title").rlike(f"(?i){search_term}"))
            matching_movies = matching_movies_df.collect()

            if not matching_movies:
                print("  Aradığınız kriterlere uygun film bulunamadı. Lütfen tekrar deneyin.")
                continue

            print("\n  Bulunan filmler:")
            for j, movie in enumerate(matching_movies[:10]):
                print(f"    {j+1}: {movie['title']}")
            
            try:
                choice = int(input("\n  Lütfen oylamak istediğiniz filmin numarasını seçin (veya 0'a basıp tekrar arayın): "))
                if choice == 0:
                    continue
                
                selected_movie = matching_movies[choice - 1]
                
                while True: # Geçerli bir puan girilene kadar tekrar et
                    rating_str = input(f"  '{selected_movie['title']}' filmine verdiğiniz puan (1-5): ")
                    try:
                        rating = float(rating_str)
                        if 1.0 <= rating <= 5.0:
                            new_user_ratings.append((new_user_id, selected_movie['movieId'], rating))
                            print(f"  '{selected_movie['title']}' için puanınız kaydedildi.")
                            break # Puan geçerli, iç döngüden çık
                        else:
                            print("  Geçersiz puan. Lütfen 1 ile 5 arasında bir değer girin.")
                    except ValueError:
                        print("  Geçersiz format. Lütfen sayısal bir değer girin (örn: 4.5).")
                
                break # Film başarıyla oylandı, ana döngüye devam et

            except (ValueError, IndexError):
                print("  Geçersiz seçim. Lütfen listedeki bir sayıyı girin.")
                continue

    # --- Oylamalar bittikten sonra tavsiyeleri üret ---
    if not new_user_ratings:
        print("\nHiç film oylanmadığı için tavsiye üretilemiyor.")
    else:
        # --- TAVSİYE 1: İÇERİK TABANLI (Tag Genome ile) ---
        highest_rated_entry = max(new_user_ratings, key=lambda item: item[2])
        highest_rated_movie_id = highest_rated_entry[1]
        highest_rated_movie_title = movies_df.filter(col("movieId") == highest_rated_movie_id).first().title
        
        print(f"\n--- TAVSİYE 1: En Beğendiğiniz Film Olan '{highest_rated_movie_title}'e Benzer Filmler (Derin İçerik Analizi) ---")
        
        content_based_recs_df = find_similar_movies(movie_id=highest_rated_movie_id, top_n=5)
        
        if content_based_recs_df is not None:
            content_based_recs_df.select("title", "genres", "similarity").show(truncate=False)

        # --- TAVSİYE 2: KİŞİSELLEŞTİRİLMİŞ VE ÇEŞİTLENDİRİLMİŞ (ALS) ---
        print("\n--- TAVSİYE 2: Oylarınıza Göre Size Özel Keşif Önerileri ---")
        
        new_ratings_schema = StructType([StructField("user", IntegerType()), StructField("item", IntegerType()), StructField("rating", FloatType())])
        new_user_ratings_df = spark.createDataFrame(data=new_user_ratings, schema=new_ratings_schema)
        enhanced_training_df = training_df.union(new_user_ratings_df)
        
        print("  Kişisel modeliniz oluşturuluyor...")
        temp_als_model = als.fit(enhanced_training_df)
        print("  Modeliniz hazır!")

        rated_movie_ids = [r[1] for r in new_user_ratings]
        user_unseen_movies_df = movies_df.filter(~col("movieId").isin(rated_movie_ids)).select(col("movieId").alias("item"), lit(new_user_id).alias("user"))
        
        predictions_df = temp_als_model.transform(user_unseen_movies_df)

        # ÇEŞİTLENDİRME ADIMI
        top_50_predictions = predictions_df.orderBy(col("prediction").desc()).limit(50)
        
        diversified_recommendations = (top_50_predictions
                                       .withColumn("random_sort", rand())
                                       .orderBy("random_sort")
                                       .limit(10)
                                       .join(movies_df, top_50_predictions.item == movies_df.movieId)
                                       .orderBy(col("prediction").desc()))

        diversified_recommendations.select("title", "genres", "prediction").show(truncate=False)

except (ValueError, IndexError):
    print("\nGeçersiz giriş. Lütfen sayısal bir değer girin.")
except Exception as e:
    print(f"\nTavsiye üretilirken bir hata oluştu: {e}")
finally:
    print("\nTavsiye işlemi tamamlandı.")


İNTERAKTİF FİLM TAVSİYE SİSTEMİNE HOŞ GELDİNİZ
Size geçici olarak '330976' kullanıcı ID'si atanmıştır.



Kaç adet film oylamak istersiniz? (Öneri: 5-10 arası):  3



--- 1. Filmi Oylama ---


Lütfen 1. filmin adını girin (örn: 'Jumanji'):  godfather



  Bulunan filmler:
    1: Godfather, The (1972)
    2: Godfather: Part II, The (1974)
    3: Godfather: Part III, The (1990)
    4: Tokyo Godfathers (2003)
    5: 3 Godfathers (1948)
    6: Last Godfather, The (2010)
    7: Disco Godfather (1979)
    8: The New Godfathers (1979)
    9: The Black Godfather (1974)
    10: Three Godfathers (1936)


...


  Bulunan filmler:
    1: Terminator 2: Judgment Day (1991)
    2: Terminator, The (1984)
    3: Exterminator, The (1980)
    4: Terminator 3: Rise of the Machines (2003)
    5: Terminator Salvation (2009)
    6: Lady Terminator (Pembalasan ratu pantai selatan) (1989)
    7: Terminator Genisys (2015)
    8: Exterminator 2 (1984)
    9: The Terminators (2009)
    10: Russian Terminator (1989)



  Lütfen oylamak istediğiniz filmin numarasını seçin (veya 0'a basıp tekrar arayın):  1
  'Terminator 2: Judgment Day (1991)' filmine verdiğiniz puan (1-5):  5


  'Terminator 2: Judgment Day (1991)' için puanınız kaydedildi.

--- TAVSİYE 1: En Beğendiğiniz Film Olan 'Terminator 2: Judgment Day (1991)'e Benzer Filmler (Derin İçerik Analizi) ---


+-------------------------------------------------+-----------------------+----------+
|title                                            |genres                 |similarity|
+-------------------------------------------------+-----------------------+----------+
|Silence of the Lambs, The (1991)                 |Crime|Horror|Thriller  |0.999756  |
|Godfather, The (1972)                            |Crime|Drama            |0.99975914|
|Star Wars: Episode VI - Return of the Jedi (1983)|Action|Adventure|Sci-Fi|0.9997523 |
|Gladiator (2000)                                 |Action|Adventure|Drama |0.99972206|
|Departed, The (2006)                             |Crime|Drama|Thriller   |0.99975246|
+-------------------------------------------------+-----------------------+----------+


--- TAVSİYE 2: Oylarınıza Göre Size Özel Keşif Önerileri ---
  Kişisel modeliniz oluşturuluyor...


  Modeliniz hazır!
+------------------------------------------------------+-----------------------+----------+
|title                                                 |genres                 |prediction|
+------------------------------------------------------+-----------------------+----------+
|SpongeBob SquarePants: Heroes of Bikini Bottom (2011) |Animation              |6.4992366 |
|Father and Sons (2003)                                |Comedy                 |5.5386534 |
|.hack Liminality In the Case of Yuki Aihara           |(no genres listed)     |5.2149563 |
|Billy Gardell: Halftime (2011)                        |Comedy                 |5.199389  |
|WWE: The American Dream: The Dusty Rhodes Story (2006)|Documentary            |5.199389  |
|The Cloud in Her Room (2020)                          |Drama                  |5.168961  |
|Song of Love (1947)                                   |Drama|Romance          |5.1455326 |
|C'est quoi la vie? (1999)                             |Drama

In [12]:
# --- 11. İnteraktif Tavsiye Sistemi (Hibrit Motor) ---
print("\n" + "="*50)
print("İNTERAKTİF FİLM TAVSİYE SİSTEMİNE HOŞ GELDİNİZ")
print("="*50)

try:
    new_user_id = ratings_df_raw.selectExpr("max(userId)").first()[0] + 1
    print(f"Size geçici olarak '{new_user_id}' kullanıcı ID'si atanmıştır.")

    new_user_ratings = []
    # (Burada önceki cevaptaki gibi kullanıcıdan çoklu oylama alan kod olduğunu varsayıyoruz)
    # Örnek olarak, sizin oyladığınız filmleri manuel olarak listeye ekleyelim:
    # new_user_ratings = [] # Eğer interaktif yapmak isterseniz bu satırı aktifleştirip alt satırı silin.
    new_user_ratings = [
        (new_user_id, 858, 4.0),   # Godfather, The (1972)
        (new_user_id, 412, 3.0),   # Heaven & Earth (1993)
        (new_user_id, 296, 5.0)    # Terminator 2: Judgment Day (1991) -> Gerçek ID'si 296 değil, örnek için Pulp Fiction'ı koydum. Gerçek T2 ID'si 589
    ]
    # Gerçek ID ile güncelleyelim:
    new_user_ratings = [
        (new_user_id, 858, 4.0),  # Godfather, The (1972)
        (new_user_id, 412, 3.0),  # Heaven & Earth (1993)
        (new_user_id, 589, 5.0)   # Terminator 2: Judgment Day (1991)
    ]

    print(f"\n{len(new_user_ratings)} adet film oylandı.")
    for _, movie_id, rating in new_user_ratings:
        title = movies_df.filter(col("movieId") == movie_id).first().title
        print(f"  - Film: {title}, Puan: {rating}")


    # --- Oylamalar bittikten sonra tavsiyeleri üret ---
    if not new_user_ratings:
        print("\nHiç film oylanmadığı için tavsiye üretilemiyor.")
    else:
        # TAVSİYE 1 (Değişmedi): En yüksek puanlı filme göre benzerler
        highest_rated_entry = max(new_user_ratings, key=lambda item: item[2])
        highest_rated_movie_id = highest_rated_entry[1]
        highest_rated_movie_title = movies_df.filter(col("movieId") == highest_rated_movie_id).first().title
        
        print(f"\n--- TAVSİYE 1: En Beğendiğiniz Film Olan '{highest_rated_movie_title}'e Benzer Filmler ---")
        content_based_recs_df = find_similar_movies(movie_id=highest_rated_movie_id, top_n=5)
        if content_based_recs_df is not None:
            content_based_recs_df.select("title", "genres").show(truncate=False)


        # --- HİBRİT TAVSİYE 2: OYLADIĞINIZ TÜM FİLMLERE GÖRE ÖNERİLER ---
        print("\n--- TAVSİYE 2: Oylarınıza Göre Harmanlanmış Özel Öneriler (Hibrit) ---")
        
        # Oylanan ve yüksek puan verilen her film için benzerleri toplayacağımız boş bir DataFrame
        all_recommendations_df = None

        # Sadece iyi puan (örn: 3.5 ve üzeri) verilen filmleri dikkate alalım
        highly_rated_movies = [r for r in new_user_ratings if r[2] >= 3.5]

        for _, movie_id, rating in highly_rated_movies:
            # Her bir film için 7 benzer film bulalım
            recs_for_one_movie = find_similar_movies(movie_id=movie_id, top_n=7)
            
            if recs_for_one_movie is None:
                continue

            # Benzerlik skorunu, kullanıcının o filme verdiği puanla ağırlıklandıralım
            # Bu sayede 5 verdiğiniz filmin benzerleri, 4 verdiğinizinkinden daha önemli olacak
            recs_for_one_movie = recs_for_one_movie.withColumn("weighted_similarity", col("similarity") * rating)
            
            if all_recommendations_df is None:
                all_recommendations_df = recs_for_one_movie
            else:
                all_recommendations_df = all_recommendations_df.union(recs_for_one_movie)
        
        if all_recommendations_df is not None:
            # Kullanıcının zaten oyladığı filmleri listeden çıkar
            rated_movie_ids = [r[1] for r in new_user_ratings]
            
            # Sonuçları gruplayıp, en yüksek ağırlıklı skoru alıp, duplikasyonları kaldıralım
            final_hybrid_recs = (all_recommendations_df
                                 .filter(~col("movieId").isin(rated_movie_ids))
                                 .groupBy("movieId", "title", "genres")
                                 .agg({"weighted_similarity": "max"})
                                 .withColumnRenamed("max(weighted_similarity)", "final_score")
                                 .orderBy(col("final_score").desc())
                                 .limit(10))
            
            final_hybrid_recs.select("title", "genres", "final_score").show(truncate=False)
        else:
            print("Size uygun hibrit tavsiye üretilemedi.")

except Exception as e:
    print(f"\nTavsiye üretilirken bir hata oluştu: {e}")
finally:
    print("\nTavsiye işlemi tamamlandı.")


İNTERAKTİF FİLM TAVSİYE SİSTEMİNE HOŞ GELDİNİZ
Size geçici olarak '330976' kullanıcı ID'si atanmıştır.

3 adet film oylandı.
  - Film: Godfather, The (1972), Puan: 4.0
  - Film: Age of Innocence, The (1993), Puan: 3.0
  - Film: Terminator 2: Judgment Day (1991), Puan: 5.0

--- TAVSİYE 1: En Beğendiğiniz Film Olan 'Terminator 2: Judgment Day (1991)'e Benzer Filmler ---


+-------------------------------------------------+-----------------------+
|title                                            |genres                 |
+-------------------------------------------------+-----------------------+
|Silence of the Lambs, The (1991)                 |Crime|Horror|Thriller  |
|Godfather, The (1972)                            |Crime|Drama            |
|Star Wars: Episode VI - Return of the Jedi (1983)|Action|Adventure|Sci-Fi|
|Gladiator (2000)                                 |Action|Adventure|Drama |
|Departed, The (2006)                             |Crime|Drama|Thriller   |
+-------------------------------------------------+-----------------------+


--- TAVSİYE 2: Oylarınıza Göre Harmanlanmış Özel Öneriler (Hibrit) ---


+---------------------------------------------------+-----------------------------------+------------------+
|title                                              |genres                             |final_score       |
+---------------------------------------------------+-----------------------------------+------------------+
|Silence of the Lambs, The (1991)                   |Crime|Horror|Thriller              |4.998779892921448 |
|Departed, The (2006)                               |Crime|Drama|Thriller               |4.998762309551239 |
|Star Wars: Episode VI - Return of the Jedi (1983)  |Action|Adventure|Sci-Fi            |4.998761415481567 |
|Gladiator (2000)                                   |Action|Adventure|Drama             |4.998610317707062 |
|Dark City (1998)                                   |Adventure|Film-Noir|Sci-Fi|Thriller|4.998478889465332 |
|Interstellar (2014)                                |Sci-Fi|IMAX                        |4.998234808444977 |
|Lives of Others, T